In [1]:
import os
import subprocess
import collections
import re
import csv
import json
import math

import pandas as pd
import numpy as np
import scipy

from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import RandomForestRegressor

import psycopg2
import requests
from imdbpie import Imdb
import nltk

import urllib
from bs4 import BeautifulSoup

import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib
matplotlib.style.use('ggplot')
%matplotlib inline

%matplotlib inline

In [2]:
# connecting to IMDB API
imdb = Imdb()
imdb = Imdb(anonymize=True)

In [3]:
# Querying top 250 movies
movies = imdb.top_250()
movies[0:4]

[{u'can_rate': True,
  u'image': {u'height': 1388,
   u'url': u'http://ia.media-imdb.com/images/M/MV5BODU4MjU4NjIwNl5BMl5BanBnXkFtZTgwMDU2MjEyMDE@._V1_.jpg',
   u'width': 933},
  u'num_votes': 1680226,
  u'rating': 9.3,
  u'tconst': u'tt0111161',
  u'title': u'The Shawshank Redemption',
  u'type': u'feature',
  u'year': u'1994'},
 {u'can_rate': True,
  u'image': {u'height': 500,
   u'url': u'http://ia.media-imdb.com/images/M/MV5BMjEyMjcyNDI4MF5BMl5BanBnXkFtZTcwMDA5Mzg3OA@@._V1_.jpg',
   u'width': 333},
  u'num_votes': 1149562,
  u'rating': 9.2,
  u'tconst': u'tt0068646',
  u'title': u'The Godfather',
  u'type': u'feature',
  u'year': u'1972'},
 {u'can_rate': True,
  u'image': {u'height': 500,
   u'url': u'http://ia.media-imdb.com/images/M/MV5BNDc2NTM3MzU1Nl5BMl5BanBnXkFtZTcwMTA5Mzg3OA@@._V1_.jpg',
   u'width': 333},
  u'num_votes': 786195,
  u'rating': 9,
  u'tconst': u'tt0071562',
  u'title': u'The Godfather: Part II',
  u'type': u'feature',
  u'year': u'1974'},
 {u'can_rate': True,
 

In [4]:
# Turn movie data into a dataframe
top = imdb.top_250()
df = pd.DataFrame(top)
df.head()

,can_rate,image,num_votes,rating,tconst,title,type,year
0,True,{u'url': u'http://ia.media-imdb.com/images/M/M...,1680226,9.3,tt0111161,The Shawshank Redemption,feature,1994
1,True,{u'url': u'http://ia.media-imdb.com/images/M/M...,1149562,9.2,tt0068646,The Godfather,feature,1972
2,True,{u'url': u'http://ia.media-imdb.com/images/M/M...,786195,9.0,tt0071562,The Godfather: Part II,feature,1974
3,True,{u'url': u'http://ia.media-imdb.com/images/M/M...,1665249,9.0,tt0468569,The Dark Knight,feature,2008
4,True,{u'url': u'http://ia.media-imdb.com/images/M/M...,859938,8.9,tt0108052,Schindler's List,feature,1993


In [5]:
# Retrieving columns that I think will be useful 
top250 = pd.DataFrame(movies)[['num_votes','rating', 'tconst', 'title', 'year']]
top250.head()

,num_votes,rating,tconst,title,year
0,1680226,9.3,tt0111161,The Shawshank Redemption,1994
1,1149562,9.2,tt0068646,The Godfather,1972
2,786195,9.0,tt0071562,The Godfather: Part II,1974
3,1665249,9.0,tt0468569,The Dark Knight,2008
4,859938,8.9,tt0108052,Schindler's List,1993


In [6]:
top250.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 250 entries, 0 to 249
Data columns (total 5 columns):
num_votes    250 non-null int64
rating       250 non-null float64
tconst       250 non-null object
title        250 non-null object
year         250 non-null object
dtypes: float64(1), int64(1), object(3)
memory usage: 9.8+ KB


In [7]:
# select the top 100 movies
top100 = top250.sort_values('rating', ascending=False)[0:100]
top100.head()

,num_votes,rating,tconst,title,year
0,1680226,9.3,tt0111161,The Shawshank Redemption,1994
1,1149562,9.2,tt0068646,The Godfather,1972
2,786195,9.0,tt0071562,The Godfather: Part II,1974
3,1665249,9.0,tt0468569,The Dark Knight,2008
6,1316943,8.9,tt0110912,Pulp Fiction,1994


In [8]:
# function to convert string to list of strings, removing whitespace
# from start and end (to use for getting lists of genres)
def strtolist(string):
    result = []
    current = ""
    for c in string:
        if c == ",":
            result.append(current.strip())
            current=""
        else:
            current+=(c)
    result.append(current.strip())
    return result

In [9]:
# create dictionary of movie id: {dictionary of genres and runtimes}
# also create list of all genres across all movies
allgenres=[]
genreruntime = {}
for mID in top100.tconst:
    url = "http://www.omdbapi.com/?i="+mID
    data = urllib.urlopen(url)
    parsed_data = json.loads(data.read())
    genres = strtolist(parsed_data["Genre"])
    runtime = parsed_data["Runtime"]
    rt = int(runtime[0:(len(runtime)-4)])
    genreruntime[mID] = {'genres': genres, 'runtime': rt}
    for genre in genres:
        if genre not in allgenres:
            allgenres.append(genre)

In [10]:
# add columns for Runtime and Genres to dataframe
top100["runtime_mins"] = np.nan
for genre in allgenres:
    top100[genre] = 0

In [11]:
# populate our new columns with values for each movie
for mID in top100.tconst:
    i = top100[top100['tconst']==mID].index
    top100.set_value(i, 'runtime_mins', genreruntime[mID]['runtime'])
    for genre in genreruntime[mID]['genres']:
        top100.set_value(i, genre, 1)

In [12]:
# look at % of top 100 that genre constitutes (sum is >100% due to 
# some movies having multiple genres). Make sure all nonzero
top100[allgenres].sum().sort_values(ascending=False)/top100.tconst.count()

Drama        0.72
Adventure    0.27
Crime        0.25
Action       0.18
Comedy       0.13
Mystery      0.13
Thriller     0.11
Biography    0.09
Sci-Fi       0.08
Animation    0.08
War          0.07
Romance      0.07
Fantasy      0.07
Family       0.05
History      0.04
Western      0.04
Horror       0.04
Film-Noir    0.03
Music        0.01
Sport        0.01
dtype: float64

In [13]:
# change scifi and film noir column headers to avoid "-" char
top100.rename(columns = {'Sci-Fi': 'Sci_Fi', 'Film-Noir': 'Film_Noir'}, inplace=True)
top100.columns

Index([   u'num_votes',       u'rating',       u'tconst',        u'title',
               u'year', u'runtime_mins',        u'Crime',        u'Drama',
             u'Action',    u'Adventure',      u'Western',    u'Biography',
            u'History',      u'Fantasy',      u'Romance',       u'Sci_Fi',
             u'Comedy',          u'War',      u'Mystery',    u'Animation',
             u'Family',     u'Thriller',       u'Horror',    u'Film_Noir',
              u'Music',        u'Sport'],
      dtype='object')

In [ ]:
# write results to a csv
top100.to_csv('top100.csv', encoding='utf-8', index=False)

In [ ]:
# use imdbpie to extract the reviews and ratings for each movie
top100reviews= {}
for mID in top100.tconst:
    reviews = imdb.get_title_reviews(mID, max_results=10000)
    revlist = []
    for review in reviews:
        revdict = {'revText': review.text, 'revRating': review.rating}
        revlist.append(revdict)
    top100reviews[mID] = revlist

In [ ]:
print len(top100reviews)

In [ ]:
# use regex to strip alphanumeric characters from each review and separately
# make list of all reviews
allReviews = []
for movie in top100reviews:
    for i in range(len(top100reviews[movie])):
        strippedText = re.sub(r'([^\s\w]|_)+', '', top100reviews[movie][i]['revText']).replace('\n', ' ')
        allReviews.append(strippedText)
        top100reviews[movie][i]['revText'] = strippedText

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
tvect = TfidfVectorizer(ngram_range = (1,2), stop_words='english', max_features=200)

In [ ]:
# train tvect on all reviews
tdm = tvect.fit_transform(allReviews)

In [ ]:
# merge the reviews and ratings by creating a 3-column dataframe 
# with movie, review text and rating

top100revDF = pd.DataFrame(columns = ['mID', 'revText', 'revRating'])

for movie in top100reviews:
    for i in range(len(top100reviews[movie])):
        review = top100reviews[movie][i]
        top100revDF = top100revDF.append(pd.DataFrame([[movie, review['revText'], 
            review['revRating']]], columns = ['mID', 'revText', 'revRating']), 
            ignore_index=True)

In [ ]:
# change mID column to tconst, to line up with the top100 table

top100revDF['tconst'] = top100revDF['mID']
del top100revDF['mID']

In [ ]:
# remove \r from reviews
top100revDF['revText'] = top100revDF['revText'].replace('\r', ' ').replace('\\n', ' ').replace('\\r',' ')

In [ ]:
# remove null rows
nonnull = top100revDF.dropna()

In [ ]:
# create a csv file 
nonnull.to_csv('reviews.csv', encoding='utf-8', index=False)

In [ ]:
%reload_ext sql

In [ ]:
%%sql 
postgresql://localhost:5432/imdb

In [ ]:
%%sql
DROP TABLE IF EXISTS top100;
CREATE TABLE top100
(num_votes integer,
rating decimal,
tconst text,
title text,
year integer,
runtime_mins numeric,
Crime integer,
Drama integer,
Action integer,
Thriller integer,
Adventure integer,
Fantasy integer,
Western integer,
Biography integer,
History integer,
Romance integer,
Mystery integer,
Sci_Fi integer,
War integer,
Comedy integer,
Animation integer,
Family integer,
Horror integer,
Film_Noir integer,
Music integer,
Sport integer
);

COPY top100 FROM
'/Users/Tamara/top100.csv'
DELIMITER ',' CSV HEADER;

In [ ]:
%%sql
DROP TABLE IF EXISTS reviews;
CREATE TABLE reviews 
(revText text,
 revRating decimal,
 tconst text
);

COPY reviews FROM 
'/Users/Tamara/reviews.csv'
DELIMITER ',' CSV HEADER;

In [ ]:
# connect to database and query the set
joined = %sql SELECT * FROM reviews JOIN top100 ON reviews."tconst"=top100."tconst";

In [ ]:
# joint the two tables
result = pd.DataFrame(joined, columns = (nonnull.columns).append(top100.columns))

In [ ]:
## code in case of problems with the SQL data
df = top100.merge(nonnull)

In [ ]:
df.columns

In [ ]:
df.rename(columns = {'rating': 'avgRating'}, inplace=True)
# drop title and tconst since they won't be used for modeling
df.drop(['tconst', 'title'], axis=1, inplace=True)

In [ ]:
df.info()

In [ ]:
# do more votes correspond with higher avg rating?
plt.scatter(df['num_votes'], df['avgRating'])
plt.xlabel("num_votes")
plt.ylabel("avgRating")

In [ ]:
# What does distribution of ratings look like?
plt.hist(df['revRating'])
plt.xlabel("score")
plt.ylabel("num reviews")

In [ ]:
# 1 seems to be most heavily used of low scores - when people dislike
# a movie they just score it at 1 rather than bother distinguishing between 1-4 (ish).
# Another interpretation is that people whose true opinion of a movie is middling
# (between, say, 2-7 rating equivalent) aren't as motivated to share their opinion as
# those who either loved or hated the movie.

In [ ]:
# look at avg rating vs runtime
plt.scatter(df['runtime_mins'], df['avgRating'])
plt.xlabel('runtime mins')
plt.ylabel('avg rating')

In [ ]:
# num votes versus year (are fewer people voting on older movies?)

plt.scatter(df['year'], df['num_votes'])

In [ ]:
df.columns

In [ ]:
# define X, y
y = df['revRating']
X = df.drop(['num_votes', 'avgRating', 'revText', 'revRating'], axis=1)

In [ ]:
y.head()

In [ ]:
# Need to add tfidf columns to the dataset
dft = pd.DataFrame(tvect.transform(df['revText']).toarray(), columns = tvect.get_feature_names())

In [ ]:
X = pd.concat([X, dft], axis = 1)

In [ ]:
X.columns

In [ ]:
# there are 2 'year' columns, one for release year and one for the word 'year' in a review. Need to rename
newColHeaders = X.columns[1:].tolist()
newColHeaders.insert(0, "ReleaseYear")

In [ ]:
X.columns = newColHeaders

In [ ]:
# build and cross validate the tree
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor
from sklearn.cross_validation import cross_val_score
from sklearn.grid_search import GridSearchCV
from sklearn.metrics import classification_report, confusion_matrix

In [ ]:
# instantiate classifier, use max_depth of 4 to control complexity
dt = DecisionTreeClassifier(max_depth = 4, random_state = rs)

In [ ]:
dt.fit(X,y)

In [ ]:
cvscores = cross_val_score(dt, X, y, n_jobs = -1)
print cvscores
print cvscores.mean()

In [ ]:
# set up param grid
param_grid = {
    'max_depth': [None,2,4,6,8,10],
    'min_samples_split': range(5,100,10),
    'max_features': [None, 10, 100]
}

In [ ]:
# instantiate gridsearchcv
gsdt = GridSearchCV(dt, param_grid, n_jobs=-1, verbose=1)

In [ ]:
gsdt.fit(X,y)

In [ ]:
gsdt.best_estimator_

In [ ]:
cvscores = cross_val_score(gsdt.best_estimator_, X, y)
print cvscores
print cvscores.mean()

In [ ]:
gsdt_pred = gsdt.predict(X)
print classification_report(y, gsdt_pred)

In [ ]:
# Build and cross-validate your decision tree regressor
dtr = DecisionTreeRegressor(random_state=rs)

In [ ]:
dtr.fit(X,y)

In [ ]:
cvscores = cross_val_score(dtr, X, y)
print cvscores
print cvscores.mean()

In [ ]:
dtr_pred = dtr.predict(X)

In [ ]:
plt.scatter(y, dtr_pred, alpha=.1)
plt.xlabel("True Rating")
plt.ylabel("Predicted Rating")

In [ ]:
# instantiate gs, use same param grid as previous to see if theres improvement
gsdtr = GridSearchCV(dtr, param_grid, n_jobs=-1, verbose=1)

In [ ]:
gsdtr.fit(X,y)

In [ ]:
gsdtr.best_estimator_

In [ ]:
cvscores = cross_val_score(gsdtr.best_estimator_, X, y)
print cvscores
print cvscores.mean()

In [ ]:
gsdtr_pred = gsdtr.predict(X)
plt.scatter(y, gsdtr_pred, alpha= .1)
plt.xlabel("True Rating")
plt.ylabel("Predicted Rating")

In [ ]:
from sklearn.ensemble import RandomForestRegressor, AdaBoostRegressor, ExtraTreesRegressor
rfr = RandomForestRegressor(random_state=rs, n_jobs=-1, verbose=1)
abr = AdaBoostRegressor(random_state=rs)
etr = ExtraTreesRegressor(random_state=rs,n_jobs=-1, verbose=1)